In [1]:
import os
%pwd

'/Users/ofotech_fitri/Documents/fitri_github/data-science-with-mlflow/research'

In [2]:
os.chdir("../")
%pwd

'/Users/ofotech_fitri/Documents/fitri_github/data-science-with-mlflow'

In [3]:
from dotenv import load_dotenv

load_dotenv()

# Access environment variables
mlflow_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow_tracking_username = os.getenv("MLFLOW_TRACKING_USERNAME")
mlflow_tracking_password = os.getenv("MLFLOW_TRACKING_PASSWORD")

# Update Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str


# Update the configuration manager in src config

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from mlProject import logger


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        model_name = self.config.model_trainer.model_name
        
        def get_model_params(model_name: str):
            if model_name in self.params:
                return self.params[model_name]
            else:
                logger.error("Model name not exist in params.yaml")
        
        params = get_model_params(model_name)
        # params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            all_params = params,
            target_column = schema.name,
            mlflow_uri = mlflow_uri
        )

        return model_evaluation_config

# Update the components

In [6]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import json
from mlProject.constants import *

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig, config_filepath = CONFIG_FILE_PATH):
        self.config = config
        self.model = read_yaml(config_filepath)


    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    def log_into_mlflow(self):
        model_name = self.model.model_trainer.model_name

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            pred = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, pred)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            metric_file_name_json = Path(self.config.metric_file_name)

            # Save the scores dictionary as JSON
            with open(metric_file_name_json, "w") as json_file:
                json.dump(scores, json_file)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                # mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
                mlflow.sklearn.log_model(model, "model", registered_model_name=f"{model_name}")
            else:
                mlflow.sklearn.log_model(model, "model")

    


# Update the pipeline

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-15 11:28:29,381: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-15 11:28:29,383: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-15 11:28:29,385: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-15 11:28:29,385: INFO: common: created directory at: artifacts]
[2023-12-15 11:28:29,386: INFO: common: created directory at: artifacts/05_model_evaluation]
[2023-12-15 11:28:29,388: INFO: common: yaml file: config/config.yaml loaded successfully]


/Users/ofotech_fitri/miniconda3/envs/mlflow/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'XGboostRegression'.
2023/12/15 11:28:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGboostRegression, version 1
Created version '1' of model 'XGboostRegression'.
